In [ ]:
from py4j.protocol import Py4JJavaError
from pyspark import SparkConf, SparkContext
import pyspark
from pyspark.sql import functions,SparkSession, SQLContext
from pyspark.sql.functions import explode, col, udf, concat_ws, from_json, lit, array, expr, size
from pyspark.sql.functions import sum as _sum
from pyspark.sql.types import *
import json
import os
import gc
from pyspark.sql.types import BooleanType
# from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
input_bucket = 's3://cs5344-twitter-project'
test_pokemon_damping_sdf = spark.read.format('csv').options(header='False').load('s3://cs5344-twitter-project/test_pokemon_damping').cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
pokemon = test_pokemon_damping_sdf.select(col("_c0").alias("user_id_str")).cache()
pokemon.createOrReplaceTempView("pokemon")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
input_bucket = 's3://cs5344-twitter-project'
path = '/{}/*.json.gz'
folder_list = ['20220122', '20220123','20220124','20220125', 
               '20220126', '20220127','20220128', '20220129',
               '20220130', '20220131', '20220201', '20220202',
               '20220203', '20220204', '20220205', '20220206',
               '20220207', '20220208', '20220209', '20220210',
               '20220211', '20220212', '20220213', '20220214',
               '20220215']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
def pre_process(folder):
    raw = spark.read.json(folder, allowBackslashEscapingAnyCharacter=True)

    user_df = raw.select(
        col("user.id_str").alias("user_id_str"),
        col("user.screen_name").alias("user_twitter_handle"),
        "user.verified",
        "user.followers_count",
        "user.friends_count",
        "user.favourites_count",
        "user.statuses_count",
        "user.created_at",
        col("created_at").alias("tweet_created_at"),
        "retweet_count"
    )

    user_df.createOrReplaceTempView("user_df")

    user_filter = spark.sql("""
    select *
        from
            (select user_id_str, 
                    user_twitter_handle, 
                    verified, 
                    followers_count, 
                    friends_count, 
                    favourites_count, 
                    statuses_count, 
                    created_at,
                    tweet_created_at,
                    rank() over(partition by user_id_str order by tweet_created_at desc) create_rank
            from user_df
            where user_id_str is not null) u
        where create_rank = 1
    """)
    user_filter.createOrReplaceTempView("user_filter")

    user_join = spark.sql("""
    select user_filter.*
    from
    user_filter
    join
    pokemon
    on user_filter.user_id_str = pokemon.user_id_str""")
    return user_join

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
folder_list[:-9:-1]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['20220215', '20220214', '20220213', '20220212', '20220211', '20220210', '20220209', '20220208']

In [ ]:
folder_list[-9:-17:-1]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['20220207', '20220206', '20220205', '20220204', '20220203', '20220202', '20220201', '20220131']

In [ ]:
folder_list[-17:-26:-1]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['20220130', '20220129', '20220128', '20220127', '20220126', '20220125', '20220124', '20220123', '20220122']

In [ ]:
# diff = pokemon.alias("diff")
pokemon_inf = None
# i = 0
for f in folder_list[:-9:-1]:
    folder = input_bucket + path.format(f)
    try:
        user_join = pre_process(folder)
        if pokemon_inf:
            pokemon_inf = pokemon_inf.union(user_join).cache()
        else:
            pokemon_inf = user_join.alias("pokemon_inf")
        del user_join
        gc.collect()
    except Py4JJavaError:
        print(folder+" failed")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

138
174
176
176
176
176
176
176

In [ ]:
for f in folder_list[-9:-17:-1]:
    folder = input_bucket + path.format(f)
    try:
        user_join = pre_process(folder)
        pokemon_inf = pokemon_inf.union(user_join).cache()
        del user_join
        gc.collect()
    except Py4JJavaError:
        print(folder+" failed")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
for f in folder_list[-17:-26:-1]:
    
    folder = input_bucket + path.format(f)
    try:
        user_join = pre_process(folder)
        pokemon_inf = pokemon_inf.union(user_join).cache()
        del user_join
        gc.collect()
    except Py4JJavaError:
        print(folder+" failed")

In [ ]:
pokemon_inf.show(1000)

In [ ]:
output_bucket = 's3://cs5344-twitter-project/'
# # pokemon_inf.toDF().write.mode('overwrite').csv(output_bucket+'pokemon_inf_24_aws') 
pokemon_inf.write.format("csv").save(output_bucket+'user_ana/pokemon_inf_0414') 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
pokemon_inf.show(1000)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+-------------------+--------+---------------+-------------+----------------+--------------+--------------------+--------------------+-----------+
|        user_id_str|user_twitter_handle|verified|followers_count|friends_count|favourites_count|statuses_count|          created_at|    tweet_created_at|create_rank|
+-------------------+-------------------+--------+---------------+-------------+----------------+--------------+--------------------+--------------------+-----------+
|1200355745600069635|       GiveAwayHost|   false|         308830|          798|            7874|         40311|Fri Nov 29 10:08:...|Tue Feb 15 21:55:...|          1|
|           34429906|     SoulRebornGame|   false|          72990|           59|             210|            99|Wed Apr 22 22:48:...|Tue Feb 15 08:44:...|          1|
|1156269110545793024|       ladyincrypto|   false|         353342|          726|            1546|         13953|Tue Jul 30 18:23:...|Tue Feb 15 06:52:...|          1

In [ ]:
#  diff = diff.join(user_join, on="user_id_str", how="left_anti")

#     if pokemon_inf:
#         pokemon_inf = pokemon_inf.union(user_join)
#     else:
#         pokemon_inf = user_join.alias("pokemon_inf")
        
        #     if diff.count() == 0:
#         print('okaykkk')
#         break

In [ ]:
# folder_list[:-6:-1]

# folder_list[-6:-11:-1]

# folder_list[-11:-16:-1]

# folder_list[-16:-21:-1]

# folder_list[-21:-26:-1]

In [ ]:
folder_list[:-15:-1]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
name 'folder_list' is not defined
Traceback (most recent call last):
NameError: name 'folder_list' is not defined



In [ ]:
folder_list[:-6:-1],folder_list[-6:-16:-1], folder_list[-16:-26:-1]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(['20220215', '20220214', '20220213', '20220212', '20220211'], ['20220210', '20220209', '20220208', '20220207', '20220206', '20220205', '20220204', '20220203', '20220202', '20220201'], ['20220131', '20220130', '20220129', '20220128', '20220127', '20220126', '20220125', '20220124', '20220123', '20220122'])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

201
204
204
204
204

In [ ]:
pokemon_inf.cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[user_id_str: string, user_twitter_handle: string, verified: boolean, followers_count: bigint, friends_count: bigint, favourites_count: bigint, statuses_count: bigint, created_at: string, tweet_created_at: string, create_rank: int]

In [ ]:
# pokemon_inf.write.format("csv").save("path/to/csv")

In [ ]:
# output_bucket = 's3://cs5344-twitter-project/'
# # pokemon_inf.toDF().write.mode('overwrite').csv(output_bucket+'pokemon_inf_24_aws') 
# pokemon_inf.write.format("csv").save(output_bucket+'pokemon_inf_24_aws') 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
# pokemon_inf.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

24

In [ ]:
diff = pokemon.alias("diff")
pokemon_inf = None
for f in folder_list[-6:-16:-1]:
    folder = input_bucket + path.format(f)
    try:
        user_join = pre_process(folder)
#         diff = diff.join(user_join, on="user_id_str", how="left_anti")
        pokemon_inf = user_join.alias("pokemon_inf")
        del user_join
        gc.collect()
    except Py4JJavaError:
        print(folder+" failed")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

138
167
167
167
167
167
167
167
167
167

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
output_bucket = 's3://cs5344-twitter-project/'
# # pokemon_inf.toDF().write.mode('overwrite').csv(output_bucket+'pokemon_inf_24_aws') 
user_join_2.write.format("csv").save(output_bucket+'user_ana/pokemon_inf_16_26_aws') 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
pokemon_inf.show(50, truncate= False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
'NoneType' object has no attribute 'show'
Traceback (most recent call last):
AttributeError: 'NoneType' object has no attribute 'show'



In [ ]:
pokemon_inf.show(50, truncate= False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+-------------------+--------+---------------+-------------+----------------+--------------+------------------------------+------------------------------+-----------+
|user_id_str        |user_twitter_handle|verified|followers_count|friends_count|favourites_count|statuses_count|created_at                    |tweet_created_at              |create_rank|
+-------------------+-------------------+--------+---------------+-------------+----------------+--------------+------------------------------+------------------------------+-----------+
|1235177017441529861|game7400           |false   |15830          |2217         |8265            |7577          |Wed Mar 04 12:15:36 +0000 2020|Tue Feb 01 05:31:43 +0000 2022|1          |
|2786677338         |denfaminicogame    |true    |419250         |3327         |8409            |44114         |Tue Sep 02 21:20:04 +0000 2014|Tue Feb 01 11:16:05 +0000 2022|1          |
|19426551           |NFL                |true    |28515096       

In [ ]:
# diff = pokemon.alias("diff")
pokemon_inf = None
for f in folder_list[-16:-26:-1]:
    folder = input_bucket + path.format(f)
    try:
        user_join_2 = pre_process(folder)
#         diff = diff.join(user_join, on="user_id_str", how="left_anti")
#         pokemon_inf = user_join.alias("pokemon_inf")
#         del user_join
        gc.collect()
    except Py4JJavaError:
        print(folder+" failed")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

187
160
160
160
160
160
160
160
160
160

In [ ]:
user_join_2.show(50, truncate= False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+-------------------+--------+---------------+-------------+----------------+--------------+------------------------------+------------------------------+-----------+
|user_id_str        |user_twitter_handle|verified|followers_count|friends_count|favourites_count|statuses_count|created_at                    |tweet_created_at              |create_rank|
+-------------------+-------------------+--------+---------------+-------------+----------------+--------------+------------------------------+------------------------------+-----------+
|1302006102364745728|strokegamexxx      |false   |175502         |1911         |1238            |67741         |Fri Sep 04 22:10:31 +0000 2020|Sat Jan 22 20:42:30 +0000 2022|1          |
|1200355745600069635|GiveAwayHost       |false   |296187         |788          |7876            |38286         |Fri Nov 29 10:08:11 +0000 2019|Sat Jan 22 20:09:25 +0000 2022|1          |
|19631395           |videogamedeals     |true    |459783         

In [ ]:
folder_list[-16:-26:-1]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['20220131', '20220130', '20220129', '20220128', '20220127', '20220126', '20220125', '20220124', '20220123', '20220122']

In [ ]:
for f in folder_list[-11:-16:-1]:
    folder = input_bucket + path.format(f)
    try:
        user_join, diff = pre_process(folder,diff)
        if diff.count() == 0:
            print('--------------------all influ have been found--------------------')
            break
        diff = diff.join(user_join, on="user_id_str", how="left_anti")
        if pokemon_inf:
            pokemon_inf = pokemon_inf.union(user_join)
        else:
            pokemon_inf = user_join.alias("pokemon_inf")
        del user_join
        gc.collect()
    except Py4JJavaError:
        print(folder+" failed")

VBox()

An error was encountered:
Session 1 unexpectedly reached final status 'dead'. See logs:
stdout: 

stderr: 
23/04/08 03:30:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/04/08 03:30:12 INFO DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at ip-172-31-37-162.ap-southeast-1.compute.internal/172.31.37.162:8032
23/04/08 03:30:12 INFO Configuration: resource-types.xml not found
23/04/08 03:30:12 INFO ResourceUtils: Unable to find 'resource-types.xml'.
23/04/08 03:30:12 INFO Client: Verifying our application has not requested more than the maximum memory capability of the cluster (12288 MB per container)
23/04/08 03:30:12 INFO Client: Will allocate AM container, with 2432 MB memory including 384 MB overhead
23/04/08 03:30:12 INFO Client: Setting up container launch context for our AM
23/04/08 03:30:12 INFO Client: Setting up the launch environment for our AM container
23/04/08 03:30:12 INFO C

In [ ]:
pokemon_inf.show()

VBox()

An error was encountered:
Session 1 unexpectedly reached final status 'dead'. See logs:
stdout: 

stderr: 
23/04/08 03:30:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/04/08 03:30:12 INFO DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at ip-172-31-37-162.ap-southeast-1.compute.internal/172.31.37.162:8032
23/04/08 03:30:12 INFO Configuration: resource-types.xml not found
23/04/08 03:30:12 INFO ResourceUtils: Unable to find 'resource-types.xml'.
23/04/08 03:30:12 INFO Client: Verifying our application has not requested more than the maximum memory capability of the cluster (12288 MB per container)
23/04/08 03:30:12 INFO Client: Will allocate AM container, with 2432 MB memory including 384 MB overhead
23/04/08 03:30:12 INFO Client: Setting up container launch context for our AM
23/04/08 03:30:12 INFO Client: Setting up the launch environment for our AM container
23/04/08 03:30:12 INFO C

In [ ]:
for f in folder_list[-16:-21:-1]:
    folder = input_bucket + path.format(f)
    try:
        user_join, diff = pre_process(folder,diff)
        if diff.count() == 0:
            print('--------------------all influ have been found--------------------')
            break
        diff = diff.join(user_join, on="user_id_str", how="left_anti")
        if pokemon_inf:
            pokemon_inf = pokemon_inf.union(user_join)
        else:
            pokemon_inf = user_join.alias("pokemon_inf")
        del user_join
        gc.collect()
    except Py4JJavaError:
        print(folder+" failed")

VBox()

An error was encountered:
Session 1 unexpectedly reached final status 'dead'. See logs:
stdout: 

stderr: 
23/04/08 03:30:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/04/08 03:30:12 INFO DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at ip-172-31-37-162.ap-southeast-1.compute.internal/172.31.37.162:8032
23/04/08 03:30:12 INFO Configuration: resource-types.xml not found
23/04/08 03:30:12 INFO ResourceUtils: Unable to find 'resource-types.xml'.
23/04/08 03:30:12 INFO Client: Verifying our application has not requested more than the maximum memory capability of the cluster (12288 MB per container)
23/04/08 03:30:12 INFO Client: Will allocate AM container, with 2432 MB memory including 384 MB overhead
23/04/08 03:30:12 INFO Client: Setting up container launch context for our AM
23/04/08 03:30:12 INFO Client: Setting up the launch environment for our AM container
23/04/08 03:30:12 INFO C

In [ ]:
pokemon_inf.show()

VBox()

An error was encountered:
Session 1 unexpectedly reached final status 'dead'. See logs:
stdout: 

stderr: 
23/04/08 03:30:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/04/08 03:30:12 INFO DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at ip-172-31-37-162.ap-southeast-1.compute.internal/172.31.37.162:8032
23/04/08 03:30:12 INFO Configuration: resource-types.xml not found
23/04/08 03:30:12 INFO ResourceUtils: Unable to find 'resource-types.xml'.
23/04/08 03:30:12 INFO Client: Verifying our application has not requested more than the maximum memory capability of the cluster (12288 MB per container)
23/04/08 03:30:12 INFO Client: Will allocate AM container, with 2432 MB memory including 384 MB overhead
23/04/08 03:30:12 INFO Client: Setting up container launch context for our AM
23/04/08 03:30:12 INFO Client: Setting up the launch environment for our AM container
23/04/08 03:30:12 INFO C

In [ ]:
for f in folder_list[-21:-26:-1]:
    folder = input_bucket + path.format(f)
    try:
        user_join, diff = pre_process(folder,diff)
        if diff.count() == 0:
            print('--------------------all influ have been found--------------------')
            break
        diff = diff.join(user_join, on="user_id_str", how="left_anti")
        if pokemon_inf:
            pokemon_inf = pokemon_inf.union(user_join)
        else:
            pokemon_inf = user_join.alias("pokemon_inf")
        del user_join
        gc.collect()
    except Py4JJavaError:
        print(folder+" failed")

VBox()

An error was encountered:
Session 1 unexpectedly reached final status 'dead'. See logs:
stdout: 

stderr: 
23/04/08 03:30:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/04/08 03:30:12 INFO DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at ip-172-31-37-162.ap-southeast-1.compute.internal/172.31.37.162:8032
23/04/08 03:30:12 INFO Configuration: resource-types.xml not found
23/04/08 03:30:12 INFO ResourceUtils: Unable to find 'resource-types.xml'.
23/04/08 03:30:12 INFO Client: Verifying our application has not requested more than the maximum memory capability of the cluster (12288 MB per container)
23/04/08 03:30:12 INFO Client: Will allocate AM container, with 2432 MB memory including 384 MB overhead
23/04/08 03:30:12 INFO Client: Setting up container launch context for our AM
23/04/08 03:30:12 INFO Client: Setting up the launch environment for our AM container
23/04/08 03:30:12 INFO C

In [ ]:
pokemon_inf.show()

VBox()

An error was encountered:
Session 1 unexpectedly reached final status 'dead'. See logs:
stdout: 

stderr: 
23/04/08 03:30:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/04/08 03:30:12 INFO DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at ip-172-31-37-162.ap-southeast-1.compute.internal/172.31.37.162:8032
23/04/08 03:30:12 INFO Configuration: resource-types.xml not found
23/04/08 03:30:12 INFO ResourceUtils: Unable to find 'resource-types.xml'.
23/04/08 03:30:12 INFO Client: Verifying our application has not requested more than the maximum memory capability of the cluster (12288 MB per container)
23/04/08 03:30:12 INFO Client: Will allocate AM container, with 2432 MB memory including 384 MB overhead
23/04/08 03:30:12 INFO Client: Setting up container launch context for our AM
23/04/08 03:30:12 INFO Client: Setting up the launch environment for our AM container
23/04/08 03:30:12 INFO C

In [ ]:
pokemon_inf.count()

VBox()

An error was encountered:
Session 1 unexpectedly reached final status 'dead'. See logs:
stdout: 

stderr: 
23/04/08 03:30:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/04/08 03:30:12 INFO DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at ip-172-31-37-162.ap-southeast-1.compute.internal/172.31.37.162:8032
23/04/08 03:30:12 INFO Configuration: resource-types.xml not found
23/04/08 03:30:12 INFO ResourceUtils: Unable to find 'resource-types.xml'.
23/04/08 03:30:12 INFO Client: Verifying our application has not requested more than the maximum memory capability of the cluster (12288 MB per container)
23/04/08 03:30:12 INFO Client: Will allocate AM container, with 2432 MB memory including 384 MB overhead
23/04/08 03:30:12 INFO Client: Setting up container launch context for our AM
23/04/08 03:30:12 INFO Client: Setting up the launch environment for our AM container
23/04/08 03:30:12 INFO C

In [ ]:
diff.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

34

In [ ]:
diff.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+
|        user_id_str|
+-------------------+
|          130447415|
|         2786677338|
|          307902310|
| 811765148595535872|
|          323091128|
|           29758446|
| 909119962572242944|
|            5162861|
| 987228577522376704|
|1470439914953011204|
|         3095001124|
|1349579108187992071|
|1441557548427927554|
|1441557548427927554|
|          252087695|
|         1149178590|
|         2597010860|
|             890891|
|1365337673074933765|
|1432825944259973123|
+-------------------+
only showing top 20 rows

In [ ]:
user_join.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
name 'user_join' is not defined
Traceback (most recent call last):
NameError: name 'user_join' is not defined



In [ ]:
pokemon_inf.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+-------------------+--------+---------------+-------------+----------------+--------------+--------------------+--------------------+-----------+--------------+-------------------------------------------------------+
|        user_id_str|user_twitter_handle|verified|followers_count|friends_count|favourites_count|statuses_count|          created_at|    tweet_created_at|create_rank|retweets_count|((retweets_count + favourites_count) / followers_count)|
+-------------------+-------------------+--------+---------------+-------------+----------------+--------------+--------------------+--------------------+-----------+--------------+-------------------------------------------------------+
|1409917553514848257|        PokemonGems|   false|          48683|          383|            6517|          1041|Tue Jun 29 16:52:...|Tue Feb 15 23:27:...|          1|             0|                                    0.13386603126348007|
|           19426551|                NFL|    tru

In [ ]:
# IER = (retweets + favorites) / followers

In [ ]:
# raw.select().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+--------------+
|retweet_count|favorite_count|
+-------------+--------------+
|            0|             0|
|            0|             0|
|            0|             0|
|            0|             0|
|            0|             0|
|            0|             0|
|            0|             0|
|            0|             0|
|            0|             0|
|            0|             0|
|            0|             0|
|            0|             0|
|            0|             0|
|            0|             0|
|            0|             0|
|            0|             0|
|            0|             0|
|            0|             0|
|            0|             0|
|            0|             0|
+-------------+--------------+
only showing top 20 rows

In [ ]:
from pyspark.sql.functions import sum, first

# Filter DataFrame to include only tweets by a particular influencer
influencer_tweets = tweets_df.filter(tweets_df.user_id == influencer_id)

# Calculate total number of retweets and favorites for the influencer's tweets
total_retweets = influencer_tweets.agg(sum("retweets")).first()[0]
total_favorites = influencer_tweets.agg(sum("favorites")).first()[0]

# Get the number of followers for the influencer
follower_count = influencer_tweets.select("followers").distinct().first()[0]

# Calculate the IER
ier = (total_retweets + total_favorites) / follower_count

In [ ]:
diff.count(), pokemon_inf.count()

In [ ]:
pokemon.count() == 0

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

False

In [ ]:
pokemon_inf.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+-------------------+--------+---------------+-------------+----------------+--------------+--------------------+--------------------+-----------+
|        user_id_str|user_twitter_handle|verified|followers_count|friends_count|favourites_count|statuses_count|          created_at|    tweet_created_at|create_rank|
+-------------------+-------------------+--------+---------------+-------------+----------------+--------------+--------------------+--------------------+-----------+
|1235177017441529861|           game7400|   false|          18430|         2215|            8277|          7677|Wed Mar 04 12:15:...|Tue Feb 15 15:23:...|          1|
|1483999454244274178|       ashward_game|   false|          54211|            4|              17|            27|Thu Jan 20 03:07:...|Tue Feb 15 09:49:...|          1|
|           19426551|                NFL|    true|       28837319|         2723|            3080|        234547|Sat Jan 24 01:28:...|Tue Feb 15 22:55:...|          1

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
res.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+-------------------+--------+---------------+-------------+----------------+--------------+--------------------+--------------------+-----------+
|        user_id_str|user_twitter_handle|verified|followers_count|friends_count|favourites_count|statuses_count|          created_at|    tweet_created_at|create_rank|
+-------------------+-------------------+--------+---------------+-------------+----------------+--------------+--------------------+--------------------+-----------+
|1235177017441529861|           game7400|   false|          18430|         2215|            8277|          7677|Wed Mar 04 12:15:...|Tue Feb 15 15:23:...|          1|
|1483999454244274178|       ashward_game|   false|          54211|            4|              17|            27|Thu Jan 20 03:07:...|Tue Feb 15 09:49:...|          1|
|           19426551|                NFL|    true|       28837319|         2723|            3080|        234547|Sat Jan 24 01:28:...|Tue Feb 15 22:55:...|          1

In [ ]:
res.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

16

In [ ]:
print("-"*20 + "user_filter" + "-"*20)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

--------------------user_filter--------------------